In [1]:
import pandas as pd
import os

import COMBINE_harmonizer
from COMBINE_harmonizer import cfg

## 01. init

In [2]:
study_name = COMBINE_harmonizer.STUDY_LH
sheet_name = COMBINE_harmonizer.SHEET_MAIN

root_dir = '..'


In [3]:
COMBINE_harmonizer.init(f'{root_dir}/config.yaml')

## 02. Dictionary

In [4]:
data_dict_filename = f'{root_dir}/Dictionary_HIE_clinical_variables.xlsx'

In [5]:
df_dict = COMBINE_harmonizer.load_data_dict(data_dict_filename, sheet_name)

In [6]:
dict_lh_columns = list(df_dict[study_name].unique())

## 03. LH

In [7]:
the_dir = cfg.config[f'{study_name}_dir']

In [8]:
filenames = os.listdir(the_dir)
filenames.sort()

In [9]:
dfs = [pd.read_csv(os.sep.join([the_dir, each_filename])) for each_filename in filenames]

### 03.1. check columns in each file

In [10]:
for idx, each_df in enumerate(dfs):
    print(f'({idx}/{len(dfs)}) filename: {filenames[idx]} columns: {list(each_df.columns)}')

(0/27) filename: lh01.csv columns: ['SITENM', 'LAST', 'FIRST', 'HTHRM_ID', 'REC_CMP', 'CENTER', 'BIRTHDT', 'BIRTHNM', 'CMP_DATE', 'CRT_DATE']
(1/27) filename: lh02.csv columns: ['HTHRM_ID', 'LH2TEMP', 'LH2COOLD', 'LH2CHROM', 'LH2CNGEN', 'LH2WGHT', 'LH2SURV', 'LH2ALLBG', 'LH2POSTN', 'LH2CNFCT', 'LH2PH', 'LH2DEFIC', 'LH2HIST', 'LH2APGA', 'LH2SEIZ', 'LH2NEXDN', 'LH2SIGN', 'LH2RESP', 'LH2SEDA', 'LH2NAME', 'LH2REAS', 'LH2CONSP', 'LH2INIT', 'REC_CMP', 'CENTER', 'LH2NENDR', 'LH2CON', 'LH2SPON', 'LH2POST', 'LH2TONE', 'LH2SUCK', 'LH2MORO', 'LH2PUPL', 'LH2HR', 'LH2NEDAT', 'LH2NETIM', 'LH2CONS', 'LH2RANDT', 'LH2RANTM', 'LH2RAGE', 'LH2LEVEL', 'LH2RANNM', 'CMP_DATE', 'CRT_DATE']
(2/27) filename: lh04.csv columns: ['HTHRM_ID', 'LH4MBRT', 'LH4PCARE', 'LH4HYPR', 'LH4HMRG', 'LH4THYR', 'LH4DIAB', 'LH4DECEL', 'LH4CORD', 'LH4UTER', 'LH4SHOUL', 'LH4PLCPX', 'LH4MHEM', 'LH4MTRMA', 'LH4MCARD', 'LH4MSEIZ', 'LH4PYRX', 'LH4CHORI', 'LH4PPATH', 'LH4CHDOC', 'LH4ANTI', 'LH4ESTIM', 'LH4LABOR', 'LH4INIT', 'REC_CMP', '

### 03.2 all columns

In [11]:
all_columns = []
for idx, each_df in enumerate(dfs):
    all_columns += list(map(lambda x: {'column': x, 'idx': idx, 'filename': filenames[idx]}, list(each_df.columns)))
df_columns = pd.DataFrame(all_columns)
df_columns['column:filename'] = df_columns.apply(lambda x: f"{x['column']}:{x['filename']}", axis=1)
df_columns['column:filename:idx'] = df_columns.apply(lambda x: f"{x['column']}:{x['filename']}:{x['idx']}", axis=1)
df_columns.sort_values(by=['filename', 'column'], inplace=True)

In [12]:
df_columns

,column,idx,filename,column:filename,column:filename:idx
6,BIRTHDT,0,lh01.csv,BIRTHDT:lh01.csv,BIRTHDT:lh01.csv:0
7,BIRTHNM,0,lh01.csv,BIRTHNM:lh01.csv,BIRTHNM:lh01.csv:0
5,CENTER,0,lh01.csv,CENTER:lh01.csv,CENTER:lh01.csv:0
8,CMP_DATE,0,lh01.csv,CMP_DATE:lh01.csv,CMP_DATE:lh01.csv:0
9,CRT_DATE,0,lh01.csv,CRT_DATE:lh01.csv,CRT_DATE:lh01.csv:0
...,...,...,...,...,...
860,LM3WSHEDL,26,lhmr03c.csv,LM3WSHEDL:lhmr03c.csv,LM3WSHEDL:lhmr03c.csv:26
861,LM3WSHEDL_code,26,lhmr03c.csv,LM3WSHEDL_code:lhmr03c.csv,LM3WSHEDL_code:lhmr03c.csv:26
862,LM3WSHEDR,26,lhmr03c.csv,LM3WSHEDR:lhmr03c.csv,LM3WSHEDR:lhmr03c.csv:26
863,LM3WSHEDR_code,26,lhmr03c.csv,LM3WSHEDR_code:lhmr03c.csv,LM3WSHEDR_code:lhmr03c.csv:26


In [13]:
df_columns_count = df_columns.groupby(['column']).agg(count=('column', 'count')).sort_values(by=['count'], ascending=False)
df_columns_count.head(20)

,count
column,
CENTER,23
HTHRM_ID,23
CMP_DATE,22
CRT_DATE,22
REC_CMP,22
Site,4
LM3ID,4
DateComplete,4
DateCreated,4


### 03.3 check HTHRM_ID

In [14]:
is_HTHRM_ID = df_columns['column'] == 'HTHRM_ID'
df_columns[is_HTHRM_ID]

,column,idx,filename,column:filename,column:filename:idx
3,HTHRM_ID,0,lh01.csv,HTHRM_ID:lh01.csv,HTHRM_ID:lh01.csv:0
10,HTHRM_ID,1,lh02.csv,HTHRM_ID:lh02.csv,HTHRM_ID:lh02.csv:1
54,HTHRM_ID,2,lh04.csv,HTHRM_ID:lh04.csv,HTHRM_ID:lh04.csv:2
104,HTHRM_ID,3,lh05.csv,HTHRM_ID:lh05.csv,HTHRM_ID:lh05.csv:3
155,HTHRM_ID,4,lh06.csv,HTHRM_ID:lh06.csv,HTHRM_ID:lh06.csv:4
221,HTHRM_ID,5,lh06a.csv,HTHRM_ID:lh06a.csv,HTHRM_ID:lh06a.csv:5
240,HTHRM_ID,6,lh06bg.csv,HTHRM_ID:lh06bg.csv,HTHRM_ID:lh06bg.csv:6
253,HTHRM_ID,7,lh06cv.csv,HTHRM_ID:lh06cv.csv,HTHRM_ID:lh06cv.csv:7
268,HTHRM_ID,8,lh06f.csv,HTHRM_ID:lh06f.csv,HTHRM_ID:lh06f.csv:8
279,HTHRM_ID,9,lh06he.csv,HTHRM_ID:lh06he.csv,HTHRM_ID:lh06he.csv:9


### 03.4 check LM3ID

In [15]:
is_LM3ID = df_columns['column'] == 'LM3ID'
df_columns[is_LM3ID]

,column,idx,filename,column:filename,column:filename:idx
697,LM3ID,23,lhmr03.csv,LM3ID:lhmr03.csv,LM3ID:lhmr03.csv:23
791,LM3ID,24,lhmr03a.csv,LM3ID:lhmr03a.csv,LM3ID:lhmr03a.csv:24
815,LM3ID,25,lhmr03b.csv,LM3ID:lhmr03b.csv,LM3ID:lhmr03b.csv:25
844,LM3ID,26,lhmr03c.csv,LM3ID:lhmr03c.csv,LM3ID:lhmr03c.csv:26


##### column not in dict

In [16]:
is_not_in_dict = df_columns['column'].isin(dict_lh_columns) == False
df_columns_not_in_dict = df_columns[is_not_in_dict]
df_columns_not_in_dict

,column,idx,filename,column:filename,column:filename:idx
8,CMP_DATE,0,lh01.csv,CMP_DATE:lh01.csv,CMP_DATE:lh01.csv:0
9,CRT_DATE,0,lh01.csv,CRT_DATE:lh01.csv,CRT_DATE:lh01.csv:0
2,FIRST,0,lh01.csv,FIRST:lh01.csv,FIRST:lh01.csv:0
1,LAST,0,lh01.csv,LAST:lh01.csv,LAST:lh01.csv:0
4,REC_CMP,0,lh01.csv,REC_CMP:lh01.csv,REC_CMP:lh01.csv:0
...,...,...,...,...,...
871,DateComplete,26,lhmr03c.csv,DateComplete:lhmr03c.csv,DateComplete:lhmr03c.csv:26
872,DateCreated,26,lhmr03c.csv,DateCreated:lhmr03c.csv,DateCreated:lhmr03c.csv:26
870,FormStatus,26,lhmr03c.csv,FormStatus:lhmr03c.csv,FormStatus:lhmr03c.csv:26
873,KeyedUser,26,lhmr03c.csv,KeyedUser:lhmr03c.csv,KeyedUser:lhmr03c.csv:26


In [17]:
columns_not_in_dict = list(df_columns_not_in_dict['column:filename:idx'].unique())
# columns_not_in_dict.sort()
columns_not_in_dict

['CMP_DATE:lh01.csv:0',
 'CRT_DATE:lh01.csv:0',
 'FIRST:lh01.csv:0',
 'LAST:lh01.csv:0',
 'REC_CMP:lh01.csv:0',
 'CMP_DATE:lh02.csv:1',
 'CRT_DATE:lh02.csv:1',
 'LH2INIT:lh02.csv:1',
 'LH2NAME:lh02.csv:1',
 'REC_CMP:lh02.csv:1',
 'CMP_DATE:lh04.csv:2',
 'CRT_DATE:lh04.csv:2',
 'LH4INIT:lh04.csv:2',
 'REC_CMP:lh04.csv:2',
 'CMP_DATE:lh05.csv:3',
 'CRT_DATE:lh05.csv:3',
 'LH5INIT:lh05.csv:3',
 'REC_CMP:lh05.csv:3',
 'CMP_DATE:lh06.csv:4',
 'CRT_DATE:lh06.csv:4',
 'REC_CMP:lh06.csv:4',
 'CMP_DATE:lh06a.csv:5',
 'CRT_DATE:lh06a.csv:5',
 'REC_CMP:lh06a.csv:5',
 'CMP_DATE:lh06bg.csv:6',
 'CRT_DATE:lh06bg.csv:6',
 'REC_CMP:lh06bg.csv:6',
 'CMP_DATE:lh06cv.csv:7',
 'CRT_DATE:lh06cv.csv:7',
 'REC_CMP:lh06cv.csv:7',
 'CMP_DATE:lh06he.csv:9',
 'CRT_DATE:lh06he.csv:9',
 'REC_CMP:lh06he.csv:9',
 'CMP_DATE:lh06ht.csv:10',
 'CRT_DATE:lh06ht.csv:10',
 'REC_CMP:lh06ht.csv:10',
 'CMP_DATE:lh06om.csv:11',
 'CRT_DATE:lh06om.csv:11',
 'REC_CMP:lh06om.csv:11',
 'CMP_DATE:lh06rs.csv:12',
 'CRT_DATE:lh06rs.cs

#### dict not in column

In [18]:
is_dict_not_in_columns = (df_dict[study_name].isin(df_columns['column']) == False) & (df_dict[study_name].isnull() == False)
df_dict_not_in_columns = df_dict[is_dict_not_in_columns]
df_dict_not_in_columns

,Category,Subcategory,Standardized_VariableNames_Dictionary,type,Variable_Description,#studies w/ this var,redcap,comment,LH,OC


## 04. show column values

In [19]:
# XXX no show due to data privacy concern.

'''
for idx, each_df in enumerate(dfs):
    for idx2, column in enumerate(each_df.columns):
        print(f'({idx}/{len(dfs)}/{filenames[idx]}) ({idx2}/{len(each_df.columns)} column: {column} value: {list(each_df[column].unique())}')
'''

"\nfor idx, each_df in enumerate(dfs):\n    for idx2, column in enumerate(each_df.columns):\n        print(f'({idx}/{len(dfs)}/{filenames[idx]}) ({idx2}/{len(each_df.columns)} column: {column} value: {list(each_df[column].unique())}')\n"